In [36]:
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
import sys
sys.path.append('/home/walaa-shaban/Documents/project/chatbot-llama3')
from src.embedding import embedding
from langchain.llms import Ollama
from langchain_core.prompts import PromptTemplate

#history
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [14]:
embedding = embedding(SentenceTransformer('all-MiniLM-L6-v2'))

/home/walaa-shaban/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/walaa-shaban/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
loader1 = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/#task-decomposition",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(class_=("post-header","post-content","post-title"))
    )
)
page1 = loader1.load()

In [18]:
loader2 = PyPDFLoader("/home/walaa-shaban/Documents/project/chatbot-llama3/input/Introduction to Machine Learning with Python.pdf")
pages2 = loader2.load_and_split()

In [19]:
loader3 = PyPDFLoader("/home/walaa-shaban/Documents/project/chatbot-llama3/input/thebook.pdf")
pages3 = loader3.load_and_split()

In [20]:
pages = page1 + pages2 + pages3

In [21]:
text_splitters_small = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)
all_splits_small = text_splitters_small.split_documents(pages)

In [22]:
len(all_splits_small)

3114

In [24]:
vector_database = Chroma.from_documents(documents=all_splits_small,embedding=embedding)

/home/walaa-shaban/anaconda3/lib/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [25]:
retriever = vector_database.as_retriever(search_type="similarity",search_kwargs={"k":4})

In [26]:
retrieved_docs = retriever.invoke("how can Using Evaluation Metrics in Model Selection")

In [27]:
retrieved_docs

[Document(page_content='299Using Evaluation Metrics in Model Selection                                                        300', metadata={'page': 6, 'source': '/home/walaa-shaban/Documents/project/chatbot-llama3/input/Introduction to Machine Learning with Python.pdf', 'start_index': 2752}),
 Document(page_content='Using Evaluation Metrics in Model Selection\nWe have discussed many evaluation methods in detail, and how to apply them given\nthe ground truth and a model. However, we often want to use metrics like AUC in\nmodel selection using GridSearchCV  or cross_val_score . Luckily scikit-learn\nprovides a very simple way to achieve this, via the scoring  argument that can be used\nin both GridSearchCV  and cross_val_score . Y ou can simply provide a string', metadata={'page': 313, 'source': '/home/walaa-shaban/Documents/project/chatbot-llama3/input/Introduction to Machine Learning with Python.pdf', 'start_index': 0}),
 Document(page_content='function used for model selection and m

In [29]:
llm = Ollama(model="llama3:latest")

In [30]:
for chunk in llm.stream("What are the main components of the autonomous agent system?"):
    print(chunk,end="",flush=True)

The Autonomous Agent System (AAS) is a complex system that involves multiple components working together to enable autonomous decision-making. The main components of an AAS can be categorized into three primary groups:

**1. Sensors and Perception**: This group includes the sensors that gather information from the environment, such as:
	* Cameras
	* LIDAR (Light Detection and Ranging)
	* Radar
	* GPS
	* Inertial Measurement Unit (IMU)
	* Accelerometers
	* Gyroscopes

These sensors provide data about the agent's surroundings, including its position, velocity, orientation, and other relevant information.

**2. Reasoning and Decision-Making**: This group includes the cognitive components that process the sensor data and make decisions:
	* Artificial Intelligence (AI) algorithms (e.g., machine learning, deep learning)
	* Rule-based systems
	* Fuzzy logic
	* Expert systems

These cognitive components analyze the sensor data, apply rules or learned patterns, and generate a response to the si

In [31]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
template = """
You are an AI bot developed , your role is to answer the users questions from the knowledge you know,
At the end of the answer thank the user.
The answer must be detailed and no less than 100 words.
if you are asked about yourself, tell the user that I am chatbot.

what to do if the answer is not encluded in the prompt or the context:
    1. apologies to the user.
    2. tell the user that you do not know the answer for the asked question.
    3. tell the user that you are specialized in the communication only.
    4. ask the user if he has more questions to ask.
    5. do not mention anything about the context.

knowledge you know:
{context}

Question: {question}

answer:
"""



In [34]:
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs , 'question': RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [35]:
for chunk in rag_chain.stream("how can using R2 in regression model?"):
    print(chunk,end="",flush=True)

When it comes to evaluating a regression model, one common metric used is the R2 score (also known as the coefficient of determination). The R2 score measures the goodness of fit of a regression model and provides a value between 0 and 1. A perfect prediction would yield an R2 score of 1, while a completely random predictor would result in an R2 score of 0.

In the context of regression modeling, using R2 can be particularly useful because it takes into account both the variance of the model's predictions and the variance of the target variable itself. This means that R2 not only measures how well the model fits the training data but also considers how much variation in the target variable is explained by the model.

In practice, using R2 as an evaluation metric for regression models can be particularly useful when:

1. Comparing different models: By calculating the R2 score for each model, you can compare their relative performance.
2. Identifying overfitting or underfitting: If your 

History

In [37]:
contextualize_q_system_prompt = """
Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone \
question which can be understood without the chat history. Do not answer the question, just reformulate it if needed and otherwise return it as is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        (MessagesPlaceholder("chat_history")),
        ("human", "{input}")
    ]
)

In [38]:
history_awar_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [39]:
qa_system_prompt = """
You are an assistant for question-answering tasks. use the following peices of retrieved context to answer the question.\
if you don't know the answer, just say that you don't know. Use four senteces maximum and keep the answer concise.\

{context}
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",qa_system_prompt), # role message pairs
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

In [40]:
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_awar_retriever,question_answer_chain)

In [41]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [42]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [43]:
conversational_rag_chain.invoke(
    {"input":"how can using R2 in regression model?"},
    config={
        "configurable": {"session_id":"123"}
    }
)["answer"]

'You can use the R2 score method for regressors to evaluate your regression model. The R2 score, also known as the coefficient of determination, is a measure of goodness of a prediction between 0 and 1, where 1 corresponds to a perfect prediction. This method is enough in most applications for evaluating regression models.'

In [44]:
conversational_rag_chain.invoke(
    {"input":"how can using it for getting score?"},
    config={
        "configurable": {"session_id":"123"}
    }
)["answer"]

'You can use the R2 score method by calling the `score` method on your regressor model. The R2 score will be returned, which measures the goodness of a prediction between 0 and 1, where 1 corresponds to a perfect prediction.'